In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

In [3]:
xls_col_info = get_excel_col_header(f"{test_data_path}\SalesData1.xlsx")
xls_col_info

{'SalesInfo': {'StoreID': {'ColumnID': 'A'},
  'SalesYear': {'ColumnID': 'B'},
  'SalesValue': {'ColumnID': 'C'},
  'StoreLocation': {'ColumnID': 'D'},
  'Owner': {'ColumnID': 'E'}},
 'RegionInfo': {'ID': {'ColumnID': 'A'}, 'Location': {'ColumnID': 'B'}},
 'StoreInfo': {'ID': {'ColumnID': 'A'},
  'RegionID': {'ColumnID': 'B'},
  'StoreOwner': {'ColumnID': 'C'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A'},
  'AverageSales': {'ColumnID': 'B'},
  'MinSales': {'ColumnID': 'C'},
  'MaxSales': {'ColumnID': 'D'}},
 'SaleGrowthReport2': {'StoreLocation-SalesYear': {'ColumnID': 'A'},
  'Year': {'ColumnID': 'B'},
  'Location': {'ColumnID': 'C'},
  'AverageSales': {'ColumnID': 'D'},
  'MinSales': {'ColumnID': 'E'},
  'MaxSales': {'ColumnID': 'F'}}}

In [4]:
xls_str = process_excel_columns_xlwings(f"{test_data_path}\SalesData1.xlsx",xls_col_info)
xls_str

{'SalesInfo': {'StoreID': {'ColumnID': 'A', 'ColumnValue': '11'},
  'SalesYear': {'ColumnID': 'B', 'ColumnValue': '2020'},
  'SalesValue': {'ColumnID': 'C', 'ColumnValue': '15789'},
  'StoreLocation': {'ColumnID': 'D',
   'ColumnValue': '=IFERROR(INDEX(RegionInfo!B:B, MATCH(INDEX(StoreInfo!B:B, MATCH(A2, StoreInfo!A:A, 0)), RegionInfo!A:A, 0)), "")'},
  'Owner': {'ColumnID': 'E',
   'ColumnValue': '=IFERROR(INDEX(StoreInfo!C:C, MATCH(A2, StoreInfo!A:A, 0)), "")'}},
 'RegionInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': '1'},
  'Location': {'ColumnID': 'B', 'ColumnValue': 'Halifax'}},
 'StoreInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': '11'},
  'RegionID': {'ColumnID': 'B', 'ColumnValue': '1'},
  'StoreOwner': {'ColumnID': 'C', 'ColumnValue': 'Ethan Carter'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A',
   'ColumnValue': '=_xlfn.UNIQUE(_xlfn._xlws.FILTER(SalesInfo!A:A, SalesInfo!A:A<>""))'},
  'AverageSales': {'ColumnID': 'B',
   'ColumnValue': '=AVERAGEIF(SalesInfo!A:A, A2, S

In [5]:
messages = [
    {"role": "system", "content": sheet_spark_prompt},
    {"role": "user", "content": f"Convert the excel logic representation to equivalent pyspark code : {xls_str}"},
  ]

In [6]:
cd_resp = generate_responses_from_inf(hf_token,messages,"Qwen/Qwen2.5-Coder-32B-Instruct",5000)

Current Message being Sent : [{'role': 'system', 'content': "\nYou are a helpful assistant to a Data Scientist who is working on a project to transform Excel Spreadsheets to PySpark Dataframes. \nThe Data Scientist has provided you with a dictionary of dictionaries which contains details about Excel spreadsheets.\nEach dictionary has only one key which is the name of the Sheet and the value is a dictionary with multiple keys.\nEach key is the name of the Column Header in the Excel Spreadsheet. The value is again a dictionary with two keys.\nThe first key is 'ColumnID' with the value being the associated ColumnID in Excel i.e. A, B, C, or etc.\nThe second key is 'ColumnValue' with the value being the associated formula for generating the column or the hardcoded value in the absence of the formula.\nThe dictionary looks something like:\n\n{{'SheetNum1': {'X': {'ColumnID': 'A', 'ColumnValue': 11},\n   'Y': {'ColumnID': 'B', 'ColumnValue': 2020},\n   'Z': {'ColumnID': 'C', 'ColumnValue': 1

In [7]:
Markdown(cd_resp)

### Explanation of Sheets

1. **SalesInfo**
   - **Hardcoded Columns:** StoreID, SalesYear, SalesValue
   - **Derived Columns:** StoreLocation, Owner

2. **RegionInfo**
   - **Hardcoded Columns:** ID, Location
   - **Derived Columns:** None

3. **StoreInfo**
   - **Hardcoded Columns:** ID, RegionID, StoreOwner
   - **Derived Columns:** None

4. **SaleGrowthReport1**
   - **Hardcoded Columns:** None
   - **Derived Columns:** StoreID, AverageSales, MinSales, MaxSales

5. **SaleGrowthReport2**
   - **Hardcoded Columns:** None
   - **Derived Columns:** StoreLocation-SalesYear, Year, Location, AverageSales, MinSales, MaxSales

### Pyspark Code

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, first, avg, min, max, uniq, concat_ws, substring, locate, length, expr

# Initialize Spark session
spark = SparkSession.builder.appName("ExcelToPySpark").getOrCreate()

# Placeholder to read Excel sheets into DataFrames
sales_info_df = spark.read.format("excel").option("header", "true").sheet("SalesInfo").load()
region_info_df = spark.read.format("excel").option("header", "true").sheet("RegionInfo").load()
store_info_df = spark.read.format("excel").option("header", "true").sheet("StoreInfo").load()

# Select only hardcoded columns from DataFrames
sales_info_df = sales_info_df.select("StoreID", "SalesYear", "SalesValue")

# Derive new columns for SalesInfo
sales_info_df = sales_info_df.join(store_info_df.withColumnRenamed("ID", "StoreID"), "StoreID", "left") \
    .join(region_info_df.withColumnRenamed("ID", "RegionID"), "RegionID", "left") \
    .withColumn("StoreLocation", when(expr("RegionInfo.Location is not null"), col("RegionInfo.Location")).otherwise("")) \
    .withColumn("Owner", when(expr("StoreInfo.StoreOwner is not null"), col("StoreInfo.StoreOwner")).otherwise("")) \
    .drop("RegionID", "RegionInfo.Location", "StoreInfo.StoreOwner")

# Derive new DataFrames for SaleGrowthReport1
sale_growth_report1_df = sales_info_df.groupBy("StoreID").agg(
    avg("SalesValue").alias("AverageSales"),  # Pseudo Code: AVERAGEIF(SalesInfo!A:A, A2, SalesInfo!C:C)
    min("SalesValue").alias("MinSales"),      # Pseudo Code: MIN(IF(SalesInfo!A:A=A2, SalesInfo!C:C))
    max("SalesValue").alias("MaxSales")       # Pseudo Code: MAX(IF(SalesInfo!A:A=A2, SalesInfo!C:C))
)

# Derive new DataFrames for SaleGrowthReport2
sale_growth_report2_df = sales_info_df.withColumn("StoreLocation-SalesYear", concat_ws("-", col("StoreLocation"), col("SalesYear"))) \
    .groupBy("StoreLocation-SalesYear").agg(
        first(substring(col("StoreLocation-SalesYear"), 1, locate("-", col("StoreLocation-SalesYear")) - 1)).alias("Location"),  # Pseudo Code: LEFT(A2, FIND("-", A2) - 1)
        first(substring(col("StoreLocation-SalesYear"), locate("-", col("StoreLocation-SalesYear")) + 1, length(col("StoreLocation-SalesYear")) - locate("-", col("StoreLocation-SalesYear")))).alias("Year"),  # Pseudo Code: RIGHT(A2, LEN(A2) - FIND("-", A2))
        avg("SalesValue").alias("AverageSales"),  # Pseudo Code: AVERAGEIFS(SalesInfo!C:C, SalesInfo!B:B, B2, SalesInfo!D:D, C2)
        min("SalesValue").alias("MinSales"),      # Pseudo Code: MINIFS(SalesInfo!C:C, SalesInfo!B:B, B2, SalesInfo!D:D, C2)
        max("SalesValue").alias("MaxSales")       # Pseudo Code: MAXIFS(SalesInfo!C:C, SalesInfo!B:B, B2, SalesInfo!D:D, C2)
    ) \
    .drop("StoreLocation-SalesYear")

# Display the DataFrames (for testing purposes)
sales_info_df.show()
sale_growth_report1_df.show()
sale_growth_report2_df.show()
```

### Explanation of Changes:
1. **Imported Required Functions:** Imported necessary functions from `pyspark.sql.functions`.
2. **Read Excel Sheets:** Used `spark.read.format("excel").option("header", "true").sheet(...).load()` to read appropriate sheets and select only hardcoded columns where necessary.
3. **Derived Columns:** Used `withColumn` and `join` to derive columns based on the given Excel formulas, with comments explaining the pseudo code logic.
4. **Aggregation:** Used `groupBy` and `agg` to aggregate data for derived columns in `SaleGrowthReport1` and `SaleGrowthReport2`.

If you need modifications, feel free to ask, and I'll update the code accordingly.